In [57]:
import numpy as np
import pandas as pd
import geopandas as gpd
import branca.colormap as cm
import folium
import fiona
import shapefile
import rasterstats
import math
import json

from folium.plugins import MarkerCluster
from folium.features import ColorLine
from folium.features import GeoJson
from shapely.geometry import mapping
from shapely.geometry import LineString

In [36]:
def make_lines(gdf, gradient, idx, geometry = 'geometry'):   
    coordinate_1 = gdf.loc[idx]['coordinates']
    coordinate_2 = gdf.loc[idx + 1]['coordinates']
    line = LineString([coordinate_1, coordinate_2])
#     start, end = [coordinate_1, coordinate_2]
#     line = LineString([start, end])
    
    data = {'gradient': gradient,
            'geometry':[line]}
    df_line = pd.DataFrame(data, columns = ['gradient', 'geometry'])
    
    return df_line

In [96]:
def route_to_shp(route_num):
    """input the number of route, then output a GeoDataFrame with gradient and geometry
    information of that route, and elevation_gradient for each line segment. 
    Also will save the route as shapefile named 'route_num'."""
    routes_shp = gpd.read_file('../data/six_routes.shp')
    route = routes_shp[routes_shp['ROUTE_NUM'] == route_num]
    
    elevation = rasterstats.point_query(route, '../data/sea_dtm_north.tif')
    elevation_gradient = np.diff(elevation[0])
    
    route_geometry = route.geometry.values # list of shapely geometries
    route_geometry = [mapping(route_geometry[0])]
    coordinates_route = route_geometry[0]['coordinates']
    linestring_route = []
    for i in range(len(coordinates_route)):
        linestring_route.append(coordinates_route[i][:2])
        linestring_route_df = pd.DataFrame()  
        linestring_route_df['coordinates'] = linestring_route
        
    df_route = pd.DataFrame(columns = ['gradient', 'geometry'])
    for idx in range(len(linestring_route_df) - 1):
        df_linestring = make_lines(linestring_route_df, elevation_gradient[idx], idx)
        df_route = pd.concat([df_route, df_linestring])
    
    gdf_route = gpd.GeoDataFrame(df_route)
    gdf_route.to_file("route_%d.shp"%route_num)
    return gdf_route

In [118]:
def route_map(route_num):
    """Visualize route_num map according to gradient. 
    e.g. route_num = 45 , shows plot of route_45."""
    gdf_route = gpd.read_file("route_%d.shp"%route_num)
    UW_coords = [47.655548, -122.303200]
    figure_size = folium.Figure(height = 400)
    route_map = folium.Map(location = UW_coords, zoom_start = 12)
    min_grade = min(gdf_route['gradient'])
    max_grade = max(gdf_route['gradient'])
    route_json = gdf_route.to_json()
    linear_map = cm.linear.YlOrBr_08.scale(min_grade, max_grade )
    route_layer = folium.GeoJson(route_json, style_function = lambda feature: {
        'color': linear_map(feature['properties']['gradient']),
        'weight': 5})
    route_layer.add_child
    route_map.add_child(linear_map)
    route_map.add_child(route_layer)
    route_map.add_to(figure_size)
    return route_map

## test function

In [116]:
route_to_shp(45)

,gradient,geometry
0,0.043378,LINESTRING (-122.3042444001078 47.650426466271...
0,0.302708,LINESTRING (-122.3042499837935 47.650414851645...
0,0.296529,LINESTRING (-122.3042778480845 47.650359884437...
0,0.530865,LINESTRING (-122.3043134540535 47.650282827138...
0,0.562183,LINESTRING (-122.304355292436 47.6501738804534...
0,0.677819,LINESTRING (-122.3043928226057 47.650076152306...
0,1.050817,LINESTRING (-122.3044705992802 47.649870595487...
0,1.089463,LINESTRING (-122.3045312243212 47.649646888555...
0,0.148095,LINESTRING (-122.3045806919458 47.649387405588...
0,0.193289,LINESTRING (-122.3046732123107 47.649315741777...


In [117]:
route_map(45)

# Below are draft.
## If the route is inside the tif map, below "elevatoin" will not show None values.
## elevation means the route_num's points elevation

In [32]:
routes_shp = gpd.read_file('../data/six_routes.shp')
route = routes_shp[routes_shp['ROUTE_NUM'] == 40]
elevation = rasterstats.point_query(route, '../data/sea_north.tif')
# elevation_gradient = np.diff(elevation[0])
elevation

[[None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  29.031083761713763,
  28.7618069345272,
  28.244608838612123,
  28.33513536736891,
  28.416196279310462,
  27.42940423022312,
  27.8275848785784,
  28.229771543192,
  28.597583308958626,
  28.51584750218976,
  29.811871980949743,
  29.09684867494339,
  28.32940878491268,
  28.268312192494868,
  29.00580596960283,
  28.464383902771107,
  28.878014400237433,
  28.673316159036656,
  28.6608753711797,
  28.756313177723438,

In [119]:
def route_map_color(route_num,cmap = 'YlOrBr_08'):
    """Visualize route_num map according to gradient. 
    e.g. route_num = 45 , shows plot of route_45."""
    gdf_route = gpd.read_file("route_%d.shp"%route_num)
    UW_coords = [47.655548, -122.303200]
    figure_size = folium.Figure(height = 400)
    route_map = folium.Map(location = UW_coords, zoom_start = 12)
    min_grade = min(gdf_route['gradient'])
    max_grade = max(gdf_route['gradient'])
    route_json = gdf_route.to_json()
    # below change the cmap.
    linear_map = cm.linear.cmap.scale(min_grade, max_grade )
    # above
    route_layer = folium.GeoJson(route_json, style_function = lambda feature: {
        'color': linear_map(feature['properties']['gradient']),
        'weight': 5})
    route_layer.add_child
    route_map.add_child(linear_map)
    route_map.add_child(route_layer)
    route_map.add_to(figure_size)
    return route_map

In [120]:
route_map_color(45)

AttributeError: '_LinearColormaps' object has no attribute 'cmap'